In [9]:
import os
import sys

parent_dir = os.path.abspath("/home/leandre/Projects/BioMoQA_Playground")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.data_pipeline.ipbes.create_ipbes_raw import loading_pipeline_from_raw

In [ ]:
datasets = loading_pipeline_from_raw(multi_label=True)

2025-07-24 08:36:53,590 - INFO - load raw positive datasets
2025-07-24 08:36:53,591 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-24 08:36:53,592 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-24 08:36:54,072 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-24 08:36:54,072 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-24 08:36:54,074 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-24 08:36:54,226 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-24 08:36:54,226 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-24 08:36:54,228 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-24 08:36:54,390 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-24 08:36:54,391 - INFO - pos_ds features for IAS : {'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='int32', id=None)}
2025-07-24 08:36:54,391 - INFO - load raw negative dataset
2025-07-24 08:36:54,392 - INFO - creating corpus dataset


Resolving data files:   0%|          | 0/2524 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-24 08:36:56,058 - INFO - dataset loaded
2025-07-24 08:36:56,060 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-24 08:36:56,060 - INFO - renaming raw positive datasets
2025-07-24 08:36:56,135 - INFO - Finished positives and negatives creation pipeline


In [11]:
datasets

([Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 5251
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 6281
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 3109
  })],
 Dataset({
     features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
     num_rows: 4227131
 }))

In [12]:
pos_ds_list=datasets[0]
neg_ds=datasets[1]

In [13]:
print(neg_ds[0])

{'id': 'https://openalex.org/W1979751022', 'display_name': 'Observations on an apparatus to illustrate the composition and resolution forces, and to measure the power obtained by certain parts of machinery operating on this principle', 'author': [{'au_id': 'https://openalex.org/A5055018967', 'au_display_name': 'W. R. Johnson', 'au_orcid': None, 'author_position': 'first', 'is_corresponding': True, 'au_affiliation_raw': '', 'institution_id': None, 'institution_display_name': None, 'institution_ror': None, 'institution_country_code': None, 'institution_type': None, 'institution_lineage': None}], 'ab': "Physical components of societies like infrastructures need biophysical resources for their construction, maintenance and use. These components, analyzed as societies' material stocks, predefine energy and raw materials and provide societal services, necessary for their functioning and for social welfare. The nexus between stocks, the resource flows and the services, is crucial for the anal

In [14]:
print(neg_ds['id'][:10])

['https://openalex.org/W1979751022', 'https://openalex.org/W1988954127', 'https://openalex.org/W2007621566', 'https://openalex.org/W2008505788', 'https://openalex.org/W2292613693', 'https://openalex.org/W2481624553', 'https://openalex.org/W4235958572', 'https://openalex.org/W4236495970', 'https://openalex.org/W4252441769', 'https://openalex.org/W2002187970']


In [15]:
import asyncio
import aiohttp
import pandas as pd
from tqdm.asyncio import tqdm_asyncio
from time import time
from typing import List

# ------------- CONFIGURATION -------------
EMAIL = "leandre.catogni@hesge.ch"
USER_AGENT = f"mailto:{EMAIL}"
BATCH_SIZE = 100
MAX_RETRIES = 3
SLEEP_BETWEEN_BATCHES = 0.5  # polite delay
TIMEOUT = 30  # request timeout
EXPORT_CSV = True
CSV_PATH = "openalex_async_results.csv"
# -----------------------------------------

# ---------- YOUR DATAFRAME HERE ----------
# neg_ds = pd.read_csv("your_ids.csv")  # Must contain 'id' column
# Example placeholder:
# neg_ds = pd.DataFrame({'id': ['https://openalex.org/W1234567890', 'https://openalex.org/W0987654321']})
# -----------------------------------------


def chunk_ids(id_list: List[str], size: int):
    """Chunk a list into batches of size `size`."""
    for i in range(0, len(id_list), size):
        yield id_list[i:i + size]


async def fetch_batch(session: aiohttp.ClientSession, batch_ids: List[str], attempt=1):
    """Fetch a batch of OpenAlex IDs with retries."""
    base_url = "https://api.openalex.org/works"
    joined_ids = "|".join(batch_ids)
    params = {
        "filter": f"openalex:{joined_ids}",
        "per-page": len(batch_ids),
        "select": "id,doi,title,publication_year"
    }

    try:
        async with session.get(base_url, params=params, timeout=TIMEOUT) as response:
            if response.status == 200:
                data = await response.json()
                return data["results"]
            else:
                print(f"❌ Error {response.status} on attempt {attempt}: {await response.text()}")
    except Exception as e:
        print(f"⚠️ Exception on attempt {attempt}: {e}")

    if attempt < MAX_RETRIES:
        await asyncio.sleep(SLEEP_BETWEEN_BATCHES * attempt)
        return await fetch_batch(session, batch_ids, attempt + 1)
    else:
        print(f"🚫 Failed to fetch after {MAX_RETRIES} attempts: {batch_ids}")
        return []


async def fetch_all_batches(all_ids: List[str]):
    """Main asynchronous fetcher."""
    connector = aiohttp.TCPConnector(limit=10)  # limit concurrent connections
    headers = {"User-Agent": USER_AGENT}

    async with aiohttp.ClientSession(headers=headers, connector=connector) as session:
        batches = list(chunk_ids(all_ids, BATCH_SIZE))
        tasks = [fetch_batch(session, batch) for batch in batches]
        all_results = await tqdm_asyncio.gather(*tasks, desc="Fetching OpenAlex")
        # Flatten results (list of lists)
        return [item for sublist in all_results for item in sublist]


async def main():
    # Clean and prepare OpenAlex IDs
    openalex_ids = [
        i.replace("https://openalex.org/", "")
        for i in neg_ds['id'] if isinstance(i, str) and i
    ]
    print(f"🔎 Preparing to fetch {len(openalex_ids)} OpenAlex entries...")

    start = time()
    results = await fetch_all_batches(openalex_ids)
    duration = time() - start
    print(f"✅ Done! Fetched {len(results)} results in {duration:.2f} seconds.")

    if EXPORT_CSV and results:
        df = pd.DataFrame(results)
        df.to_csv(CSV_PATH, index=False)
        print(f"📁 Saved to {CSV_PATH}")


# Run it
asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
clean_pos=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is None and batch['Item Type'][i] == 'journalArticle' and (batch['title'][i] is not None or batch['abstract'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

Filter (num_proc=20):   0%|          | 0/5251 [00:00<?, ? examples/s]

In [ ]:
print(clean_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 328
})


In [ ]:
clean_pos_a=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is not None and batch['Item Type'][i] == 'journalArticle' for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

Filter (num_proc=20):   0%|          | 0/5251 [00:00<?, ? examples/s]

In [ ]:
print(clean_pos_a)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 3829
})


In [ ]:
clean_pos_b=clean_pos_a.filter(lambda batch : [(batch['abstract'][i] is not None and batch['title'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(clean_pos_b)

Filter (num_proc=20):   0%|          | 0/3829 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 2253
})


In [ ]:
none_abs=clean_pos_a.filter(lambda batch : [batch['abstract'][i] is None or batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs)

Filter (num_proc=20):   0%|          | 0/3829 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 1576
})


In [ ]:
print(none_abs[2])

{'doi': 'https://doi.org/10.1890/02-5002', 'title': 'Fire and grazing impacts on plant diversity and alien plant invasions in the southern Sierra Nevada', 'abstract': None, 'Item Type': 'journalArticle', 'Publication Year': 2003}


In [ ]:
none_abs_instance=neg_ds.filter(lambda batch : [batch['display_name'][i] == 'Germination and early growth of Ailanthus< and tulip poplar in three levels of forest disturbance' if batch['display_name'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs_instance)

Filter (num_proc=20):   0%|          | 0/4227131 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [ ]:
pos_doi_set=set(pos_ds_list[0]['doi'])
pos_doi_set.remove(None)
none_abs_instance=neg_ds.filter(lambda batch : [any(batch['doi'][i].endswith(p_doi) for p_doi in pos_doi_set) if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=30)

Filter (num_proc=30):   0%|          | 0/4227131 [00:00<?, ? examples/s]

Process ForkPoolWorker-124:
Process ForkPoolWorker-138:
Process ForkPoolWorker-148:
Process ForkPoolWorker-149:
Process ForkPoolWorker-150:
Process ForkPoolWorker-145:
Process ForkPoolWorker-123:
Process ForkPoolWorker-140:
Process ForkPoolWorker-122:
Process ForkPoolWorker-139:
Process ForkPoolWorker-130:
Process ForkPoolWorker-136:
Process ForkPoolWorker-125:
Process ForkPoolWorker-126:
Process ForkPoolWorker-128:
Process ForkPoolWorker-127:
Process ForkPoolWorker-137:
Process ForkPoolWorker-135:
Process ForkPoolWorker-129:
Process ForkPoolWorker-121:
Process ForkPoolWorker-146:
Traceback (most recent call last):
Process ForkPoolWorker-131:
Process ForkPoolWorker-132:
  File "/home/leandre/Projects/BioMoQA_Playground/.venv/lib/python3.11/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-147:
Traceback (most recent call last):
  File "/home/leandre/Projects/BioMoQA

In [ ]:
print(len(pos_doi_set))

3861


In [ ]:
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 941
})


In [ ]:
print(neg_ds['doi'][0:10])

['https://doi.org/10.1016/s0016-0032(29)90208-8', 'https://doi.org/10.1016/s0016-0032(29)90638-4', 'https://doi.org/10.1098/rstl.1829.0026', 'https://doi.org/10.1016/s0016-0032(29)90698-0', 'https://doi.org/10.1080/14786442908675166', 'https://doi.org/10.5962/bhl.title.109375', 'https://doi.org/10.1016/s0016-0032(29)90539-1', 'https://doi.org/10.1016/s0016-0032(29)90045-4', 'https://doi.org/10.1016/s0016-0032(29)90465-8', 'https://doi.org/10.1098/rstl.1816.0012']


In [ ]:
print(neg_ds.filter(lambda batch : [batch['id'][i].endswith('W2142524821') if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [ ]:
print(neg_ds['id'][0])

https://openalex.org/W1979751022


In [ ]:
recent_pos=clean_pos_b.filter(lambda batch : [batch["Publication Year"][i]>=2021 if batch["Publication Year"][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(recent_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 158
})


In [ ]:
from src.data_pipeline.ipbes.fetch import fetch_crossref_metadata, get_metadata

params = {
    'mailto': 'leandre.catogni@hesge.ch'
}
filters = {
}
get_metadata(fetch_crossref_metadata('https://doi.org/10.1890/02-5002',params=params,filters=filters))



Record fetched successfully:
Abstract: Patterns of native and alien plant diversity in response to disturbance were examined along an elevational gradient in blue oak savanna, chaparral, and coniferous forests. Total species richness, alien species richness, and alien cover declined with elevation, at scales from 1 to 1000 m<jats:sup>2</jats:sup>. We found no support for the hypothesis that community diversity inhibits alien invasion. At the 1‐m<jats:sup>2</jats:sup> point scale, where we would expect competitive interactions between the largely herbaceous flora to be most intense, alien species richness as well as alien cover increased with increasing native species richness in all communities. This suggests that aliens are limited not by the number of native competitors, but by resources that affect establishment of both natives and aliens.</jats:p><jats:p>Blue oak savannas were heavily dominated by alien species and consistently had more alien than native species at the 1‐m<jats:sup

('Patterns of native and alien plant diversity in response to disturbance were examined along an elevational gradient in blue oak savanna, chaparral, and coniferous forests. Total species richness, alien species richness, and alien cover declined with elevation, at scales from 1 to 1000 m<jats:sup>2</jats:sup>. We found no support for the hypothesis that community diversity inhibits alien invasion. At the 1‐m<jats:sup>2</jats:sup> point scale, where we would expect competitive interactions between the largely herbaceous flora to be most intense, alien species richness as well as alien cover increased with increasing native species richness in all communities. This suggests that aliens are limited not by the number of native competitors, but by resources that affect establishment of both natives and aliens.</jats:p><jats:p>Blue oak savannas were heavily dominated by alien species and consistently had more alien than native species at the 1‐m<jats:sup>2</jats:sup> scale. All of these ali

In [ ]:
get_metadata(fetch_crossref_metadata('10.1890/02-5002',params=params,filters=filters))

Record fetched successfully:
Abstract: Patterns of native and alien plant diversity in response to disturbance were examined along an elevational gradient in blue oak savanna, chaparral, and coniferous forests. Total species richness, alien species richness, and alien cover declined with elevation, at scales from 1 to 1000 m<jats:sup>2</jats:sup>. We found no support for the hypothesis that community diversity inhibits alien invasion. At the 1‐m<jats:sup>2</jats:sup> point scale, where we would expect competitive interactions between the largely herbaceous flora to be most intense, alien species richness as well as alien cover increased with increasing native species richness in all communities. This suggests that aliens are limited not by the number of native competitors, but by resources that affect establishment of both natives and aliens.</jats:p><jats:p>Blue oak savannas were heavily dominated by alien species and consistently had more alien than native species at the 1‐m<jats:sup

('Patterns of native and alien plant diversity in response to disturbance were examined along an elevational gradient in blue oak savanna, chaparral, and coniferous forests. Total species richness, alien species richness, and alien cover declined with elevation, at scales from 1 to 1000 m<jats:sup>2</jats:sup>. We found no support for the hypothesis that community diversity inhibits alien invasion. At the 1‐m<jats:sup>2</jats:sup> point scale, where we would expect competitive interactions between the largely herbaceous flora to be most intense, alien species richness as well as alien cover increased with increasing native species richness in all communities. This suggests that aliens are limited not by the number of native competitors, but by resources that affect establishment of both natives and aliens.</jats:p><jats:p>Blue oak savannas were heavily dominated by alien species and consistently had more alien than native species at the 1‐m<jats:sup>2</jats:sup> scale. All of these ali

In [ ]:
from src.data_pipeline.ipbes.preprocess_ipbes import data_pipeline

clean_ds,folds_per_run=data_pipeline(2,1,balance_coeff=1,multi_label=True,fill_metadata=False)
print(clean_ds)
print(clean_ds.features)

2025-07-22 14:44:53,086 - INFO - load raw positive datasets
2025-07-22 14:44:53,088 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-22 14:44:53,089 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-22 14:44:53,528 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-22 14:44:53,528 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-22 14:44:53,530 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-22 14:44:53,688 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-22 14:44:53,688 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-22 14:44:53,689 - INFO - 128


...

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-22 14:44:55,441 - INFO - dataset loaded
2025-07-22 14:44:55,442 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-22 14:44:55,442 - INFO - renaming raw positive datasets
2025-07-22 14:44:55,522 - INFO - Finished positives and negatives creation pipeline


pos_combined Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 8701
})
Filtering out rows with no abstracts or DOI...
Size of the dataset before cleaning: 4235832
Applying clean_filter...


2025-07-22 14:44:58,244 - INFO - Dataset size after cleaing : 4851
2025-07-22 14:44:59,118 - INFO - Fold 1:
2025-07-22 14:44:59,119 - INFO -   Train label distribution: {(1, 0, 0): 0.9153439153439153, (0, 1, 0): 0.07054673721340388, (1, 1, 0): 0.007642563198118754, (1, 0, 1): 0.003527336860670194, (1, 1, 1): 0.001763668430335097, (0, 1, 1): 0.0011757789535567313}
2025-07-22 14:44:59,119 - INFO -   Test label distribution: {(1, 0, 0): 0.45683601817430813, (0, 0, 1): 0.30813713341594384, (0, 1, 0): 0.21396117306897977, (0, 1, 1): 0.00950020652622883, (1, 0, 1): 0.004956629491945477, (1, 1, 0): 0.004543577034283354, (1, 1, 1): 0.0020652622883106154}
2025-07-22 14:44:59,235 - INFO - Fold 2:
2025-07-22 14:44:59,236 - INFO -   Train label distribution: {(1, 0, 0): 0.9198113207547169, (0, 1, 0): 0.06898584905660378, (1, 0, 1): 0.0047169811320754715, (1, 1, 0): 0.004127358490566038, (0, 1, 1): 0.0011792452830188679, (1, 1, 1): 0.0011792452830188679}
2025-07-22 14:44:59,237 - INFO -   Test labe

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'id', 'display_name', 'author', 'ab', 'topics', 'author_abbr', 'IAS', 'SUA', 'VA'],
    num_rows: 4851
})
{'doi': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'abstract': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='float64', id=None), 'id': Value(dtype='string', id=None), 'display_name': Value(dtype='string', id=None), 'author': [{'au_id': Value(dtype='string', id=None), 'au_display_name': Value(dtype='string', id=None), 'au_orcid': Value(dtype='string', id=None), 'author_position': Value(dtype='string', id=None), 'is_corresponding': Value(dtype='bool', id=None), 'au_affiliation_raw': Value(dtype='string', id=None), 'institution_id': Value(dtype='string', id=None), 'institution_display_name': Value(dtype='string', id=None), 'institution_ror': Value(dtype='string', id=None), 'institution_country_code': Value(d

In [ ]:

print(clean_ds.filter(lambda batch : [batch['doi'][i] is None and (batch['IAS']==1 or batch['SUA']==1 or batch['VA']==1 ) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['abstract'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.to_pandas()['Item Type'].unique())



Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'id', 'display_name', 'author', 'ab', 'topics', 'author_abbr', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})
Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'id', 'display_name', 'author', 'ab', 'topics', 'author_abbr', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})
Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'id', 'display_name', 'author', 'ab', 'topics', 'author_abbr', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})
['journalArticle']
